In [3]:
# NHSRC PHC SUPPLY CHAIN - FEATURE ENGINEERING FOR FORECASTING
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🔮 NHSRC PHC FEATURE ENGINEERING FOR FORECASTING")
print("=" * 60)

# 1️⃣ LOAD CLEANED DATA
print("📥 1. Loading Cleaned Data...")
df = pd.read_csv("data/cleaned_inventory.csv", parse_dates=["date", "batch_expiry_date"])
print(f"   Loaded {len(df):,} records")
print(f"   Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"   Unique SKUs: {df['sku_id'].nunique()}")

# 2️⃣ CREATE CONTINUOUS TIME SERIES PER SKU
print("\n📅 2. Creating Continuous Daily Time Series...")

# FIRST: Aggregate by SKU and Date (sum across wards/batches)
print("   Aggregating data by SKU and Date...")
df_aggregated = df.groupby(['sku_id', 'date']).agg({
    'units_used': 'sum',
    'on_hand': 'last',  # Take the latest stock level for that day
    'stock_in_transit': 'sum',
    'lead_time_days': 'mean',
    'price_per_unit': 'mean',
    'is_holiday': 'max',
    'patient_admissions': 'sum',
    'ved_category': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown',
    'fsn_category': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown',
    'days_cover': 'last',
    'expiry_days_remaining': 'min',  # Most urgent expiry
    'expiry_risk_bucket': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown'
}).reset_index()

print(f"   After aggregation: {len(df_aggregated):,} records (was {len(df):,})")

# Create continuous date range
date_range = pd.date_range(start=df_aggregated['date'].min(), 
                           end=df_aggregated['date'].max(), 
                           freq='D')
print(f"   Continuous date range: {len(date_range)} days")

# Create continuous series for each SKU
continuous_frames = []

for sku in df_aggregated['sku_id'].unique():
    sku_data = df_aggregated[df_aggregated['sku_id'] == sku].copy()
    
    # Set date as index for reindexing
    sku_data_indexed = sku_data.set_index('date')
    
    # Reindex to continuous dates
    sku_continuous = sku_data_indexed.reindex(date_range)
    
    # Fill SKU identifier
    sku_continuous['sku_id'] = sku
    
    # Get other SKU metadata from original data
    sku_meta = df[df['sku_id'] == sku].iloc[0]
    sku_continuous['sku_name'] = sku_meta['sku_name']
    sku_continuous['facility_id'] = sku_meta['facility_id']
    sku_continuous['facility_name'] = sku_meta['facility_name']
    
    continuous_frames.append(sku_continuous)

# Combine all SKU continuous series
df_continuous = pd.concat(continuous_frames)

# Reset index to make date a column
df_continuous = df_continuous.reset_index().rename(columns={'index': 'date'})

print(f"   After creating continuous series: {len(df_continuous):,} records")
print(f"   Date range now continuous: {df_continuous['date'].min().date()} to {df_continuous['date'].max().date()}")
print(f"   Expected records: {df_continuous['sku_id'].nunique()} SKUs × {len(date_range)} days = {df_continuous['sku_id'].nunique() * len(date_range):,}")

# 3️⃣ FILL MISSING VALUES CAREFULLY
print("\n🔧 3. Filling Missing Values...")

# Fill consumption with 0 for missing days (no consumption recorded)
df_continuous["units_used"] = df_continuous["units_used"].fillna(0)

# Forward fill stock levels (carry last known stock forward)
df_continuous["on_hand"] = df_continuous.groupby("sku_id")["on_hand"].fillna(method="ffill")

# Fill other numeric columns with 0 or appropriate values
numeric_cols = ['stock_in_transit', 'lead_time_days', 'price_per_unit', 
                'is_holiday', 'patient_admissions', 'days_cover', 
                'expiry_days_remaining']
for col in numeric_cols:
    if col in df_continuous.columns:
        df_continuous[col] = df_continuous[col].fillna(0)

# Fill categorical columns with mode or first value
categorical_cols = ['ward_id', 'batch_id', 'ved_category', 'fsn_category', 
                    'expiry_risk_bucket', 'is_outlier']
for col in categorical_cols:
    if col in df_continuous.columns:
        # Fill with most common value per SKU
        df_continuous[col] = df_continuous.groupby('sku_id')[col].transform(
            lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown')
        )

print(f"   Missing values remaining: {df_continuous.isna().sum().sum()}")

# 4️⃣ ADD TIME-BASED FEATURES
print("\n⏰ 4. Adding Time-Based Features...")

df_continuous["day_of_week"] = df_continuous["date"].dt.dayofweek  # Monday=0, Sunday=6
df_continuous["is_weekend"] = df_continuous["day_of_week"].isin([5, 6]).astype(int)
df_continuous["day_of_month"] = df_continuous["date"].dt.day
df_continuous["month"] = df_continuous["date"].dt.month
df_continuous["week_of_year"] = df_continuous["date"].dt.isocalendar().week

print("   Added: day_of_week, is_weekend, day_of_month, month, week_of_year")

# 5️⃣ ADD LAG FEATURES (CORE FORECAST INPUTS)
print("\n📊 5. Adding Lag Features...")

# Sort by SKU and date for proper shifting
df_continuous = df_continuous.sort_values(['sku_id', 'date'])

for lag in [1, 3, 7, 14, 30]:
    df_continuous[f"lag_{lag}"] = df_continuous.groupby("sku_id")["units_used"].shift(lag)
    print(f"   Added lag_{lag}")

# 6️⃣ ADD ROLLING STATISTICAL FEATURES
print("\n📈 6. Adding Rolling Statistical Features...")

# Calculate rolling features
for win in [7, 14, 30]:
    df_continuous[f"rolling_mean_{win}"] = df_continuous.groupby("sku_id")["units_used"].transform(
        lambda x: x.rolling(window=win, min_periods=1).mean()
    )
    df_continuous[f"rolling_std_{win}"] = df_continuous.groupby("sku_id")["units_used"].transform(
        lambda x: x.rolling(window=win, min_periods=1).std()
    )
    print(f"   Added rolling_mean_{win}, rolling_std_{win}")

# Calculate rolling coefficient of variation (stability metric)
for win in [7, 14]:
    df_continuous[f"rolling_cv_{win}"] = (
        df_continuous[f"rolling_std_{win}"] / df_continuous[f"rolling_mean_{win}"].replace(0, 1)
    ).round(3)

# 7️⃣ MERGE MASTER FILE SIGNALS
print("\n🏥 7. Merging Master File Data...")

try:
    master = pd.read_csv("data/medicine_master.csv")
    
    # Standardize column names for merge
    master.columns = master.columns.str.strip().str.lower().str.replace(" ", "_")
    
    # Merge master data
    df_continuous = df_continuous.merge(
        master[['sku_id', 'ved_category', 'fsn_category', 'unit', 'strength']], 
        on='sku_id', 
        how='left',
        suffixes=('', '_master')
    )
    
    # Use master categories if available
    for col in ['ved_category', 'fsn_category']:
        if f'{col}_master' in df_continuous.columns:
            df_continuous[col] = df_continuous[f'{col}_master'].combine_first(df_continuous[col])
            df_continuous = df_continuous.drop(columns=[f'{col}_master'])
    
    print("   Master data merged successfully")
except Exception as e:
    print(f"   ⚠️  Could not merge master data: {e}")

# 8️⃣ ADD DERIVED FEATURES FOR FORECASTING
print("\n🧮 8. Adding Derived Forecasting Features...")

# Consumption trend (slope of last 7 days)
def calculate_trend(series):
    if len(series) >= 7:
        x = np.arange(len(series[-7:]))
        y = series[-7:].values
        slope = np.polyfit(x, y, 1)[0]
        return round(slope, 3)
    return 0

df_continuous['consumption_trend_7d'] = df_continuous.groupby('sku_id')['units_used'].transform(
    lambda x: x.rolling(window=7).apply(calculate_trend, raw=False)
)

# Day-over-day percentage change
df_continuous['day_over_day_pct_change'] = df_continuous.groupby('sku_id')['units_used'].pct_change().replace([np.inf, -np.inf], 0).fillna(0).round(3)

# 9️⃣ FIX ANY REMAINING MISSING VALUES
print("\n🧹 9. Final Data Cleaning...")

# Fill any remaining NaN values
initial_nan = df_continuous.isna().sum().sum()
df_continuous = df_continuous.fillna(0)
final_nan = df_continuous.isna().sum().sum()

print(f"   Fixed {initial_nan} remaining NaN values")
print(f"   Total NaN after cleaning: {final_nan}")

# 1️⃣0️⃣ SAVE OUTPUTS
print("\n💾 10. Saving Output Files...")

# Save forecast-ready time series
output_path_ts = "data/forecast_ready_timeseries.csv"
df_continuous.to_csv(output_path_ts, index=False)
print(f"   ✅ Saved: {output_path_ts}")
print(f"   Records: {len(df_continuous):,}")
print(f"   Columns: {len(df_continuous.columns)}")

# Create SKU-level features summary
sku_features = df_continuous.groupby("sku_id").agg({
    "units_used": ["mean", "std", "count"],
    "ved_category": lambda x: x.mode()[0] if not x.mode().empty else "Unknown",
    "fsn_category": lambda x: x.mode()[0] if not x.mode().empty else "Unknown",
    "rolling_mean_7": "last",
    "rolling_std_7": "last",
    "rolling_cv_7": "last"
}).round(3)

sku_features.columns = ["ADC", "ADC_std", "observation_days", "VED_category", 
                        "FSN_category", "recent_mean_7d", "recent_std_7d", "recent_cv_7d"]

sku_features = sku_features.reset_index()

# Add days since last consumption (important for slow movers)
last_consumption = df_continuous[df_continuous['units_used'] > 0].groupby('sku_id')['date'].max()
sku_features['days_since_last_consumption'] = sku_features['sku_id'].map(
    lambda x: (df_continuous['date'].max() - last_consumption.get(x, df_continuous['date'].min())).days
    if x in last_consumption else 999
)

output_path_sku = "data/sku_level_features.csv"
sku_features.to_csv(output_path_sku, index=False)
print(f"   ✅ Saved: {output_path_sku}")
print(f"   SKUs: {len(sku_features)}")

# 1️⃣1️⃣ FINAL SUMMARY
print("\n" + "="*60)
print("📊 FINAL FEATURE ENGINEERING SUMMARY")
print("="*60)

print(f"📅 Time Series Data:")
print(f"   Total records: {len(df_continuous):,}")
print(f"   Date range: {df_continuous['date'].min().date()} to {df_continuous['date'].max().date()}")
print(f"   Unique SKUs: {df_continuous['sku_id'].nunique()}")
print(f"   Total features created: {len(df_continuous.columns)}")

print(f"\n🔮 Forecasting-Ready Features:")
feature_types = {
    "Time Features": ["day_of_week", "is_weekend", "day_of_month", "month", "week_of_year"],
    "Lag Features": [f"lag_{lag}" for lag in [1, 3, 7, 14, 30]],
    "Rolling Stats": [f"rolling_mean_{win}" for win in [7, 14, 30]] + 
                    [f"rolling_std_{win}" for win in [7, 14, 30]] +
                    [f"rolling_cv_{win}" for win in [7, 14]],
    "Derived Features": ["consumption_trend_7d", "day_over_day_pct_change"]
}

for feature_type, features in feature_types.items():
    actual_features = [f for f in features if f in df_continuous.columns]
    print(f"   {feature_type}: {len(actual_features)} features")

print(f"\n🎯 Ready for AI Forecasting Models!")
print("="*60)

🔮 NHSRC PHC FEATURE ENGINEERING FOR FORECASTING
📥 1. Loading Cleaned Data...
   Loaded 6,480 records
   Date range: 2024-01-01 to 2024-06-28
   Unique SKUs: 12

📅 2. Creating Continuous Daily Time Series...
   Aggregating data by SKU and Date...
   After aggregation: 2,160 records (was 6,480)
   Continuous date range: 180 days
   After creating continuous series: 2,160 records
   Date range now continuous: 2024-01-01 to 2024-06-28
   Expected records: 12 SKUs × 180 days = 2,160

🔧 3. Filling Missing Values...
   Missing values remaining: 0

⏰ 4. Adding Time-Based Features...
   Added: day_of_week, is_weekend, day_of_month, month, week_of_year

📊 5. Adding Lag Features...
   Added lag_1
   Added lag_3
   Added lag_7
   Added lag_14
   Added lag_30

📈 6. Adding Rolling Statistical Features...
   Added rolling_mean_7, rolling_std_7
   Added rolling_mean_14, rolling_std_14
   Added rolling_mean_30, rolling_std_30

🏥 7. Merging Master File Data...
   ⚠️  Could not merge master data: "['stre

In [7]:
import pandas as pd
df = pd.read_csv('data/forecast_ready_timeseries.csv')
print('🔹 FIRST 10 ROWS OF FORECAST_READY_TIMESERIES.CSV:')
print(df.head(10).to_string())
print()
print(f'🔹 Number of rows: {len(df):,}')
print(f'🔹 Unique SKU count: {df["sku_id"].nunique()}')

🔹 FIRST 10 ROWS OF FORECAST_READY_TIMESERIES.CSV:
         date  sku_id  units_used  on_hand  stock_in_transit  lead_time_days  price_per_unit  is_holiday  patient_admissions ved_category fsn_category  days_cover  expiry_days_remaining expiry_risk_bucket                  sku_name facility_id     facility_name  day_of_week  is_weekend  day_of_month  month  week_of_year  lag_1  lag_3  lag_7  lag_14  lag_30  rolling_mean_7  rolling_std_7  rolling_mean_14  rolling_std_14  rolling_mean_30  rolling_std_30  rolling_cv_7  rolling_cv_14  consumption_trend_7d  day_over_day_pct_change
0  2024-01-01  MED001          93      211               157        5.333333       23.346667           0                 235        Vital         Fast   25.912281                    180   MEDIUM (90-180d)  Paracetamol Tablet 500mg      FAC001  Urban PHC Kalyan            0           0             1      1             1    0.0    0.0    0.0     0.0     0.0       93.000000       0.000000        93.000000        0.0000

In [9]:
import pandas as pd
df = pd.read_csv('data/forecast_ready_timeseries.csv')
print('🔹 FIRST 10 ROWS OF FORECAST_READY_TIMESERIES.CSV:')
print(df.head(10).to_string())
print()
print(f'🔹 Number of rows: {len(df):,}')
print(f'🔹 Unique SKU count: {df["sku_id"].nunique()}')

🔹 FIRST 10 ROWS OF FORECAST_READY_TIMESERIES.CSV:
         date  sku_id  units_used  on_hand  stock_in_transit  lead_time_days  price_per_unit  is_holiday  patient_admissions ved_category fsn_category  days_cover  expiry_days_remaining expiry_risk_bucket                  sku_name facility_id     facility_name  day_of_week  is_weekend  day_of_month  month  week_of_year  lag_1  lag_3  lag_7  lag_14  lag_30  rolling_mean_7  rolling_std_7  rolling_mean_14  rolling_std_14  rolling_mean_30  rolling_std_30  rolling_cv_7  rolling_cv_14  consumption_trend_7d  day_over_day_pct_change
0  01-01-2024  MED001          93      211               157        5.333333       23.346667           0                 235        Vital         Fast   25.912281                    180   MEDIUM (90-180d)  Paracetamol Tablet 500mg      FAC001  Urban PHC Kalyan            0           0             1      1             1      0      0      0       0       0       93.000000       0.000000        93.000000        0.0000